In [1]:
%matplotlib inline
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import DataLoader, TensorDataset, Subset, Dataset
from sklearn.model_selection import train_test_split, KFold
from PIL import Image
import shutil
import pandas
from torchvision.io import read_image

cudnn.benchmark = True
plt.ion()   # interactive mode
#https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

/home/justin.rossiter/.conda/envs/hfrl/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
def test_model(model, dataset, labels):
    since = time.time()

    # Each epoch has a training and validation phase
    #model.eval()

    running_corrects = 0

    # Iterate over data.
    for idx in range(len(labels)):
        
        image = Image.fromarray(dataset[:, :, idx], mode = 'L').resize((320, 320))
        label = labels[idx]
        if not isinstance(label, list):
            label = [label]
        # zero the parameter gradients
        #optimizer.zero_grad()

        # forward
        output = model(image, size=320)
        predictions = output.pandas().xyxy[0].sort_values('xmin').loc[:, "class"]
        predictions_char_arr = [str(prediction) if prediction != 10 else 'u' for prediction in predictions]
        predictions_str = "".join(predictions_char_arr)
        print("Number of predictions: ", len(predictions_str), "Prediction: ", predictions_str)
        running_corrects += (predictions == label)  
        #Calculate accuracy for each label; do average (varies by size of label): microaverage (use if imbalance)
        #Calculate accuracy for each specific prediction: macroaverage (try this)
        #output.print()
        #_, preds = torch.max(outputs, 1)
        #print(preds)
        #loss = criterion(output, label)

        # statistics
        #running_loss += loss.item() * inputs.size(0)
        #running_corrects += torch.sum(preds == labels.data)

    #loss = running_loss / dataset_test_size
    acc = running_corrects / len(labels)

    print(f'Acc: {acc:.4f}')

    time_elapsed = time.time() - since
    print(f'Testing complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')

    return model

In [21]:
preceding_path = ""
dataset = np.load(os.path.join(preceding_path, 'data_test.npy'))
print(len(dataset))
dataset = dataset.reshape((300, 300, len(dataset)))
labels = np.load(os.path.join(preceding_path, 't_test.npy'))
model_ft = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt', force_reload=True)

1807


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /home/justin.rossiter/.cache/torch/hub/master.zip
YOLOv5 🚀 2022-12-2 Python-3.9.13 torch-1.12.0 CPU

Fusing layers... 
Model summary: 157 layers, 7039792 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [22]:
test_model(model_ft, dataset, labels)

        xmin        ymin        xmax        ymax  confidence  class  \
0  78.154945  129.712082  257.281647  176.149765    0.904738      3   

            name  
0  negative sign  
Number of predictions:  1 Prediction:  3
        xmin      ymin        xmax        ymax  confidence  class         name
0  10.074173  7.611801  316.180176  292.989807    0.934571      1  square root
Number of predictions:  1 Prediction:  1
        xmin       ymin        xmax        ymax  confidence  class name
0  78.764641  77.663513  165.311371  204.019348    0.943059      8   pi
Number of predictions:  1 Prediction:  8
        xmin       ymin        xmax        ymax  confidence  class     name
0  51.555077  86.341354  243.474976  283.427917    0.622288      7  product
Number of predictions:  1 Prediction:  7
        xmin       ymin        xmax        ymax  confidence  class  \
0  63.121529  35.531586  212.905426  236.104126    0.939163      1   

          name  
0  square root  
Number of predictions:  1 

ValueError: ('Lengths must match to compare', (2,), (1,))